# Notes 

## We cannot formulate the inequality assumptions in the pinning control problems as a semidefinite programming problem

Consider the network given as 
$$
\dot{x}_i = f(x_i) + \epsilon \sum_{j = 1}^n \xi_{ij} P x_j \quad i = 1, 2, \ldots, n
$$
where $f: \mathbb{R}^d \mapsto \mathbb{R}^d$ is the function corresponding to the node dynamics and the diagonal matrix $P = diag(p_1, p_2, \ldots, p_d)$ is the inner coupling matrix. 

In the pinning control analysis of this network, we assume that there exist a symmetric matrix $K$ such that 
$$
(x - y)^T (f(x) - f(y)) \leq (x-y)^T K P (x - y) \quad \forall x, y \in \mathbb{R}^d \tag{1}
$$
Here the node dynamics function $f$ and the inner coupling matrix $P$ determines $K$. At first using optimization techniques such as semi-definite programming (SDP) can be used the matrix $K$. The problem can be formulated as a feasibility problem subjected a linear matrix inequality (LMI). An LMI is given as 
$$
L(x) = F_0 + \sum_{i = 1}^m x_i F_i
$$
where $x = [x_1, x_2, \ldots, x_m]$ is the optimization variable and $F_i = F_i^T, i = 1, 2, \ldots, m$ are the constant symmetric matrices. 

The problem here is that the inequality in (1) cannot be formulated as and LMI because of the term $f(x) - f(y)$. This implies that the variables $x$ and $y$ must be optimization variables, which in turn makes the optimization problem not semi-definite programming.


## The problem is not feasible for full synchronization of networks using pinning control 

Consider the network given above. For the full synhronization of the network the condition
$$
\Omega = \theta I_d + \epsilon (\Xi - A) < 0 \tag{2}
$$ 
where $\theta = \lambda_{max}\left((K + K^T) / 2\right)$, $A = diag(\alpha_1, \alpha_2, \ldots, \alpha_l, 0, 0, \ldots, 0)$. Thus the matrix $A$ is diagonal matrix with first $l$ diagonal entries are positive. $l$ is the number of pinned nodes.  

We have an LMI in (2) and we can find the matrix $A$ by solving a feasiblity problem(an optimization problem with no objective function) subjected to and LMI condition. So we have 
$$
\begin{aligned} 
    \text{minimmize } & 0 \\ 
    \text{s. t} \quad  & \Gamma - \tilde{A} \leq 0
\end{aligned}
$$
where $\Gamma = \theta I_d - \epsilon A$ and $\tilde{A} = \epsilon A$. However, the problem is not feasible as can be seen from the below script.

In [2]:
using LightGraphs
using JuMP 
using SCS
using LinearAlgebra
using Test 

# Network settings 
numnodes = 10 
dimnodes = 3
numpins = 8
θ = 10. 
ϵ = 1. 
graph = star_graph(numnodes)
Ξ = -collect(-laplacian_matrix(graph))
Γ = θ * I(numnodes) - ϵ * Ξ

# LMI solution 
model = Model(SCS.Optimizer) 
set_silent(model)
@variable(model, A[1 : numnodes, 1 : numnodes], PSD) 
@SDconstraint(model, Γ - A ≤ 0)    # Ω = Γ - A ≤ 0
@SDconstraint(model, A ≥ 0 )    # A ≥ 0 
for i in numpins + 1 : numnodes 
    @constraint(model, A[i, i] == 0)
end 
for i = 1 : numnodes
    for j in 1 : numnodes 
        if i ≠ j 
            # Off diagonal elements are zero.
            @constraint(model, A[i, j] == 0)
        end 
    end
end
optimize!(model)
A = value.(A)
@show termination_status(model)

termination_status(model) = MathOptInterface.INFEASIBLE


INFEASIBLE::TerminationStatusCode = 2

Note that the first $l$ diagonal entries of $A$ is nonzero and this condition makes the problem infeasible. If these constraints are deleted, then the problem is feasible agian and an optimal solution can be found. This can be seen from the script below. 

In [7]:
using LightGraphs
using JuMP 
using SCS
using LinearAlgebra
using Test 

# Network settings 
numnodes = 10 
dimnodes = 3
numpins = 8
θ = 10. 
ϵ = 1. 
graph = star_graph(numnodes)
Ξ = -collect(-laplacian_matrix(graph))
Γ = θ * I(numnodes) - ϵ * Ξ

# LMI solution 
model = Model(SCS.Optimizer) 
set_silent(model)
@variable(model, A[1 : numnodes, 1 : numnodes], PSD) 
@SDconstraint(model, Γ - A ≤ 0)    # Ω = Γ - A ≤ 0
@SDconstraint(model, A ≥ 0 )    # A ≥ 0 
# for i in numpins + 1 : numnodes 
#     @constraint(model, A[i, i] == 0)
# end 
for i = 1 : numnodes
    for j in 1 : numnodes 
        if i ≠ j 
            # Off diagonal elements are zero.
            @constraint(model, A[i, j] == 0)
        end 
    end
end
optimize!(model)
A = value.(A)
@show termination_status(model)

Ω = Γ  - A 
@show issymmetric(Ω)
@show all(eigvals(Ω) .< 0)

termination_status(model) = MathOptInterface.OPTIMAL
issymmetric(Ω) = true
all(eigvals(Ω) .< 0) = true


true